In [16]:
# import librart
import sklearn
import numpy as np
import pandas as pd
import pickle
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import scipy
from scipy.sparse import csr_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KernelDensity
import math

In [38]:
# use doc2vec
# use doc2vec
# use doc2vec
# use doc2vec

# import data
train_df = pd.read_csv("./project_data_files/book_rating_train.csv")
test_df = pd.read_csv("./project_data_files/book_rating_test.csv")



# process vector features
train_name_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/train_name_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
train_authors_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/train_authors_doc2vec20.csv", index_col = False, delimiter = ',', header=None)
train_desc_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/train_desc_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
other_features_df = train_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language'])
train_name_features.columns = [f"name_{i}" for i in range(train_name_features.shape[1])]
train_authors_features.columns = [f"authors_{i}" for i in range(train_authors_features.shape[1])]
train_desc_features.columns = [f"desc_{i}" for i in range(train_desc_features.shape[1])]
train_features = pd.concat([train_name_features, train_authors_features, train_desc_features, other_features_df], axis=1)




# use doc2vec
# use doc2vec
# use doc2vec
# use doc2vec

# process test features
test_name_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/test_name_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
test_authors_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/test_authors_doc2vec20.csv", index_col = False, delimiter = ',', header=None)
test_desc_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/test_desc_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
test_other_features_df = test_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language'])
test_name_features.columns = [f"name_{i}" for i in range(train_name_features.shape[1])]
test_authors_features.columns = [f"authors_{i}" for i in range(train_authors_features.shape[1])]
test_desc_features.columns = [f"desc_{i}" for i in range(train_desc_features.shape[1])]
test_features = pd.concat([test_name_features, test_authors_features, test_desc_features, test_other_features_df], axis=1)
# split train and testing dataset
X_train, X_val, y_train, y_val = train_test_split(train_features, train_df["rating_label"], test_size=0.2, random_state=42)


In [39]:
# This function is for calculate probablity of all lables
# From workshop
def calc_prior(data): 
    prior_prob = {}
    labels = data.iloc[:,-1]
    unique_labels, counts = np.unique(labels, return_counts=True)
    
    n = counts.sum()

    for i in range(len(unique_labels)):
        prior_prob[unique_labels[i]] = (counts[i] / n).round(2)

    return prior_prob

In [6]:
# This function is for calculate mean and standard deviation
def calc_mean_sd(feature_lst):
    mean = feature_lst.mean()
    sum = 0
    for i in feature_lst:
        sum += (math.pow(i-mean,2))
    sd = math.sqrt(sum/(len(feature_lst)-1))
    return mean,sd

In [7]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground
# truth labels

def evaluate_pvc(predictions, true_labels):
    POSITIVELABEL = "classical"
    NAGETIVELABEL = "pop"
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    total = len(predictions)
    for i in range(total):
        if predictions[i] == true_labels[i] and predictions[i] == POSITIVELABEL:
            tp += 1
        elif predictions[i] == true_labels[i] and predictions[i] == NAGETIVELABEL:
            tn += 1
        elif predictions[i] != true_labels[i] and predictions[i] == POSITIVELABEL:
            fp += 1
        elif predictions[i] != true_labels[i] and predictions[i] == NAGETIVELABEL:
            fn += 1
    accuracy = (tp+tn) / total
    precision = tp / (tp+fp)
    recall = tp / (tp+fn)
    return accuracy,precision,recall

In [40]:
# function for calculate accuracy,precision,recall
def evaluate_multiclass(predictions, true_labels, prior_prob):
    labels = list(prior_prob.keys())
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for i in labels:
        positive_label = i
        nagetive_label = [n for n in labels if n != positive_label]
        for j in range(len(predictions)):
            if predictions[j] == true_labels[j] and predictions[j] == positive_label:
                tp += 1
            elif predictions[j] == true_labels[j] and (predictions[j] in nagetive_label):
                tn += 1
            elif predictions[j] != true_labels[j] and predictions[j] == positive_label:
                fp += 1
            elif predictions[j] != true_labels[j] and (predictions[j] in nagetive_label):
                fn += 1
    total = tp+tn+fp+fn
    try:
        accuracy = (tp+tn) / total
    except:
        print("Error: total eqaul to 0")
    try:
        precision = tp / (tp+fp)
    except:
        print("Error: tp+fp eqaul to 0")
    try:
        recall = tp / (tp+fn)
    except:
        print("Error: tp+fn eqaul to 0")
    return accuracy,precision,recall

In [30]:
# function for calc feature prob
def calc_feature_prob_kde(feature_values,feature_value,hyper):
    sum = 0
    for i in feature_values:
        power = (-1/2)*math.pow((feature_value-i)/hyper,2)
        prob_f = (1/(hyper*math.sqrt(2*math.pi)))*(math.exp(power))
        sum += prob_f
    if(len(feature_values) == 0):
        return 0
    else:
        return sum/len(feature_values)
    

In [32]:
# function for predict label by KDE
def predict_kde(data, prior_prob,hyper):
    argmax_labels = []
    for _, instance in data.iterrows():
        max_prob = -99999
        max_label = None
        for label in prior_prob:
            prob = math.log(prior_prob[label])
            label_bool = data.iloc[:,-1] == label
            cur_label_df = data[label_bool]
            for feature in cur_label_df.iloc[:, :-1]:
                feature_prob = calc_feature_prob_kde(cur_label_df[feature],instance[feature],hyper)
                # underflow
                if (feature_prob == 0):
                    prob += 0
                else:
                    prob += math.log(feature_prob)
            if(prob > max_prob):
                max_prob = prob
                max_label = label
        argmax_labels.append(max_label)
    return argmax_labels

In [11]:
# This function should calculat prior probabilities and likelihoods from the training data and using
# them to build a naive Bayes model
def train_kde(data):
    # Calculate prior probabilities
    prior_prob = calc_prior(data)

    return prior_prob

In [12]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

,name_0,name_1,name_2,name_3,name_4,name_5,name_6,name_7,name_8,name_9,name_10,name_11,name_12,name_13,name_14,name_15,name_16,name_17,name_18,name_19,name_20,name_21,name_22,name_23,name_24,name_25,name_26,name_27,name_28,name_29,name_30,name_31,name_32,name_33,name_34,name_35,name_36,name_37,name_38,name_39,name_40,name_41,name_42,name_43,name_44,name_45,name_46,name_47,name_48,name_49,name_50,name_51,name_52,name_53,name_54,name_55,name_56,name_57,name_58,name_59,name_60,name_61,name_62,name_63,name_64,name_65,name_66,name_67,name_68,name_69,name_70,name_71,name_72,name_73,name_74,name_75,name_76,name_77,name_78,name_79,name_80,name_81,name_82,name_83,name_84,name_85,name_86,name_87,name_88,name_89,name_90,name_91,name_92,name_93,name_94,name_95,name_96,name_97,name_98,name_99,authors_0,authors_1,authors_2,authors_3,authors_4,authors_5,authors_6,authors_7,authors_8,authors_9,authors_10,authors_11,authors_12,authors_13,authors_14,authors_15,authors_16,authors_17,authors_18,authors_19,desc_0,desc_1,desc_2,desc_3,desc_4,desc_5,desc_6,desc_7,desc_8,desc_9,desc_10,desc_11,desc_12,desc_13,desc_14,desc_15,desc_16,desc_17,desc_18,desc_19,desc_20,desc_21,desc_22,desc_23,desc_24,desc_25,desc_26,desc_27,desc_28,desc_29,desc_30,desc_31,desc_32,desc_33,desc_34,desc_35,desc_36,desc_37,desc_38,desc_39,desc_40,desc_41,desc_42,desc_43,desc_44,desc_45,desc_46,desc_47,desc_48,desc_49,desc_50,desc_51,desc_52,desc_53,desc_54,desc_55,desc_56,desc_57,desc_58,desc_59,desc_60,desc_61,desc_62,desc_63,desc_64,desc_65,desc_66,desc_67,desc_68,desc_69,desc_70,desc_71,desc_72,desc_73,desc_74,desc_75,desc_76,desc_77,desc_78,desc_79,desc_80,desc_81,desc_82,desc_83,desc_84,desc_85,desc_86,desc_87,desc_88,desc_89,desc_90,desc_91,desc_92,desc_93,desc_94,desc_95,desc_96,desc_97,desc_98,desc_99,PublishYear,PublishMonth,PublishDay,pagesNumber
0,0.029333,-0.043927,0.102285,-0.121823,-0.016668,-0.137888,-0.059163,0.216625,-0.136272,-0.133504,-0.184495,-0.041870,-0.117257,-0.096045,0.077705,-0.191333,-0.043447,-0.047646,-0.218763,-0.100768,0.039723,0.089830,0.071220,0.006881,-0.086392,0.091715,0.057683,0.035317,-0.134711,0.053463,0.079191,0.003965,-0.057969,-0.049322,-0.097515,0.188969,0.012724,-0.041169,-0.150028,-0.044269,0.020912,-0.008203,-0.055666,-0.080331,0.105137,-0.201640,-0.007454,-0.078108,-0.154042,0.144147,0.028559,-0.087055,0.177242,-0.165604,-0.126360,0.100349,-0.025688,-0.148631,-0.014751,-0.037349,0.233187,-0.063687,0.147748,0.046032,0.063705,0.189816,0.129602,0.115044,-0.280869,-0.095834,-0.008613,-0.046109,0.020528,0.073146,0.061113,0.296201,0.015175,0.009826,-0.028021,-0.063215,-0.065596,0.053740,-0.044922,0.090794,-0.000980,-0.108602,0.162690,0.005096,0.069834,0.081681,0.046344,-0.109673,0.085691,0.025971,0.310225,0.056881,-0.186346,-0.001683,0.124991,-0.023987,-0.130635,-0.043828,0.181181,0.063403,-0.106636,-0.094421,-0.017305,0.205563,-0.263943,-0.010923,0.285141,0.044055,0.119470,0.052096,0.047779,0.100110,0.264468,-0.121419,-0.270422,-0.278139,0.811110,0.467299,0.553123,0.821670,0.529499,-0.761879,0.228721,-0.229102,-0.487880,-1.577984,0.265104,-0.401263,-0.228452,0.533100,0.323726,-0.165955,0.019214,0.076596,-0.400782,0.651151,0.676844,-0.252340,0.629294,-0.159156,-0.139988,1.122390,-0.335090,0.431367,-0.483010,-1.168252,0.047535,0.419483,-0.379594,0.276552,0.182941,-0.197267,-0.115007,0.053232,-0.209201,-0.271192,-1.121499,0.134982,-1.016577,0.197332,0.111117,-0.275350,-0.568298,0.503730,0.064979,1.398692,-1.070305,0.250332,0.808608,0.070277,-0.266485,0.648117,-0.447036,-0.305482,-0.197907,0.974465,-0.333268,-0.482068,1.083330,0.444313,-0.702738,1.481041,0.196953,1.326692,-0.173125,0.019137,1.618636,0.527408,-0.775952,0.039248,0.388727,-0.198067,-0.277779,-0.280158,-0.087489,-0.442265,-0.566304,-0.259881,1.423394,0.540603,-0.988442,-0.997691,0.354802,0.472195,0.211415,-0.360119,0.992751,1.270977,-0.217944,-0.823663,0.761966,0.643117,-1.399525,0.674203,-0.769568,0.170481,1993,1,1,118
1,0.138439,-0.003614,-0.002051,-0.061709,-0.035443,-0.10

In [43]:
# predict pvc_test by kde 
prior_prob= train_kde(train_features)
print(prior_prob)
# for choose a btter hyperparameter
i=2
prediction = predict_kde(test_features,prior_prob,i)



{3.0: 0.25, 4.0: 0.7, 5.0: 0.04}


ZeroDivisionError: division by zero

In [46]:
output_df = pd.DataFrame({'rating_label': prediction})
output_df.index += 1
output_df.index.name = 'id'
output_df.to_csv('./predictions.csv')

In [ ]:
true_labels = test_features.iloc[:,-1].tolist()
accuracy,precision,recall = evaluate_pvc(prediction,true_labels)
print("bandwidth:",i, "accuracy:",accuracy,"precision",precision,"recall",recall)